<a href="https://colab.research.google.com/github/Shobana-1206/Email-Spam-Detector/blob/main/22cs159.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/narumiruna/efficientnet-pytorch.git


Cloning into 'efficientnet-pytorch'...
remote: Enumerating objects: 876, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 876 (delta 168), reused 208 (delta 121), pack-reused 572 (from 1)
Receiving objects: 100% (876/876), 293.73 KiB | 1.67 MiB/s, done.
Resolving deltas: 100% (479/479), done.


In [ ]:
!pip install torch torchvision

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"varunkarthi","key":"7fae157991a89f17e0736de3cd3e57fa"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d uciml/zoo-animal-classification -p datasets --unzip

Dataset URL: https://www.kaggle.com/datasets/uciml/zoo-animal-classification
License(s): DbCL-1.0
  0% 0.00/1.85k [00:00<?, ?B/s]
100% 1.85k/1.85k [00:00<00:00, 5.95MB/s]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim


data = pd.read_csv('datasets/zoo.csv')


X = data.drop(columns=['animal_name', 'class_type'])
y = data['class_type']


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
class ZooAnimalDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


train_dataset = ZooAnimalDataset(X_train_scaled, y_train)
test_dataset = ZooAnimalDataset(X_test_scaled, y_test)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


input_dim = X_train_scaled.shape[1]
num_classes = len(label_encoder.classes_)
model = MLP(input_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader:
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

Epoch [1/20], Loss: 2.041816790898641
Validation Accuracy: 9.52%
Epoch [2/20], Loss: 1.9825186729431152
Validation Accuracy: 9.52%
Epoch [3/20], Loss: 1.9370214541753132
Validation Accuracy: 9.52%
Epoch [4/20], Loss: 1.8813248078028362
Validation Accuracy: 61.90%
Epoch [5/20], Loss: 1.831894834836324
Validation Accuracy: 76.19%
Epoch [6/20], Loss: 1.7677478790283203
Validation Accuracy: 76.19%
Epoch [7/20], Loss: 1.7101104656855266
Validation Accuracy: 76.19%
Epoch [8/20], Loss: 1.6583785613377888
Validation Accuracy: 76.19%
Epoch [9/20], Loss: 1.5813052256902058
Validation Accuracy: 76.19%
Epoch [10/20], Loss: 1.5044951836268108
Validation Accuracy: 80.95%
Epoch [11/20], Loss: 1.4210679133733113
Validation Accuracy: 80.95%
Epoch [12/20], Loss: 1.3646257321039836
Validation Accuracy: 80.95%
Epoch [13/20], Loss: 1.2740726470947266
Validation Accuracy: 80.95%
Epoch [14/20], Loss: 1.1657133499781291
Validation Accuracy: 80.95%
Epoch [15/20], Loss: 1.0617645581563313
Validation Accuracy: 8